In [27]:
import path
import pandas as pd
from modules.pre_processing.CSV_PreProcessor import load_and_preprocess_from_path_label
import tensorflow as tf


def convert_yolo_to_rcnn(df):
    df = df.copy()
    df["x_min"] = df["x_center"] - df["width"] / 2
    df["y_min"] = df["y_center"] - df["height"] / 2
    df["x_max"] = df["x_center"] + df["width"] / 2
    df["y_max"] = df["y_center"] + df["height"] / 2
    df = df.drop(columns=["x_center", "y_center", "width", "height"])
    return df


def load_data(df):
    image_paths = df["image_path"].values
    labels = df.drop(columns="image_path")
    labels_dict = {col: tf.constant(labels[col].values) for col in labels.columns}
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels_dict))
    dataset = dataset.map(load_and_preprocess_from_path_label)
    return dataset

In [28]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


def create_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(416, 416, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(units=128, activation="relu"))
    model.add(
        Dense(units=6, activation="sigmoid", name="dense_3")
    )  # Update the output layer name and number of units

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    return model

In [29]:
model = create_model()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [30]:
import path
from modules.pre_processing.CSV_PreProcessor import load_csv_from_path

x, Y = load_csv_from_path("../../datasets/csv/mapping/prepared.csv")
Y.reset_index(drop=True, inplace=True)

In [31]:
x = convert_yolo_to_rcnn(x)
Y = convert_yolo_to_rcnn(Y)

In [32]:
import path
import numpy as np

import tensorflow as tf

train_dataset = load_data(x)
test_dataset = load_data(Y)

In [33]:
train_dataset = test_dataset.map(lambda x, y: (tf.reshape(x, [-1, 416, 416, 3]), y))
test_dataset = test_dataset.map(lambda x, y: (tf.reshape(x, [-1, 416, 416, 3]), y))

In [34]:
history = model.fit(train_dataset, epochs=20, validation_data=test_dataset)

Epoch 1/20


2023-06-26 02:34:29.506376: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype double and shape [24026]
	 [[{{node Placeholder/_5}}]]


ValueError: in user code:

    File "/home/skira21/University of Wales Trinity St. Davids/MSc/Year 4/Project/Autonomous-Vehicles-Motorcycle-Safety/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/skira21/University of Wales Trinity St. Davids/MSc/Year 4/Project/Autonomous-Vehicles-Motorcycle-Safety/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/skira21/University of Wales Trinity St. Davids/MSc/Year 4/Project/Autonomous-Vehicles-Motorcycle-Safety/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/skira21/University of Wales Trinity St. Davids/MSc/Year 4/Project/Autonomous-Vehicles-Motorcycle-Safety/venv/lib/python3.9/site-packages/keras/engine/training.py", line

In [ ]:
import numpy as np
from modules.pre_processing.CSV_PreProcessor import (
    load_csv_from_path,
    load_and_preprocess_from_path_label,
)
from yolov8.tf import YOLOv8
import tensorflow as tf

# Load CSV data
x, Y = load_csv_from_path("../../datasets/csv/mapping/prepared.csv")

# Get class labels
train_labels = np.array(x["class"].values)
test_labels = np.array(Y["class"].values)

# Create dataset
train_dataset = tf.data.Dataset.from_tensor_slices(
    (x["image_path"].values, train_labels)
)
test_dataset = tf.data.Dataset.from_tensor_slices((Y["image_path"].values, test_labels))

# Load and preprocess images
train_dataset = train_dataset.map(load_and_preprocess_from_path_label)
test_dataset = test_dataset.map(load_and_preprocess_from_path_label)

# Shuffle and batch the dataset
train_dataset = train_dataset.shuffle(1024).batch(32)
test_dataset = test_dataset.batch(32)

# Configuring the model
num_classes = len(
    np.unique(train_labels)
)  # Assuming this is the number of unique classes in your data
model = YOLOv8(input_shape=(416, 416, 3), num_classes=num_classes)

# Compiling the model (you might want to check the loss function in the documentation of YOLOv8 you're using)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Training the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

ModuleNotFoundError: No module named 'yolov8.tf'